In [28]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [29]:
def box_score_table_reader(path, start_season, end_season, file_end):
    
    all_df = pd.read_csv(path + 'NBA_' + str(start_season) + file_end)
    all_df['season'] = start_season
    
    for i in range(start_season + 1, end_season):
        df = pd.read_csv(path + 'NBA_' + str(i) + file_end)
        df['season'] = i 
        all_df = pd.concat([all_df, df])
    
    return(all_df)

In [30]:
file_end = '_games.csv'
path = 'C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/'
df = box_score_table_reader(path, 2014, 2022, file_end)

In [31]:
df.head(2)

,Unnamed: 0,starter,Starters,player_id,MP,FG,FGA,FG%,3P,3PA,...,STL,BLK,TOV,PF,PTS,+/-,team,team_id,game_id,season
0,0,1,LeBron James,/players/j/jamesle01.html,38:01,5.0,11.0,0.455,0.0,1.0,...,1.0,0.0,2.0,0.0,17.0,8.0,Miami Heat,/teams/MIA/2014.html,201310290MIA.html,2014
1,1,1,Dwyane Wade,/players/w/wadedw01.html,36:07,5.0,13.0,0.385,1.0,1.0,...,1.0,1.0,3.0,4.0,13.0,2.0,Miami Heat,/teams/MIA/2014.html,201310290MIA.html,2014


In [32]:
df.dtypes

Unnamed: 0      int64
starter         int64
Starters       object
player_id      object
MP             object
FG            float64
FGA           float64
FG%           float64
3P            float64
3PA           float64
3P%           float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PTS           float64
+/-           float64
team           object
team_id        object
game_id        object
season          int64
dtype: object

In [33]:
df.MP = df.MP.replace('Did Not Play', '00:00').replace('Did Not Dress', '00:00').replace('Not With Team', '00:00').replace('Player Suspended', '00:00')

df['played_min'] = df.MP.str.split(':', expand = True)[0]
df['played_sec'] = df.MP.str.split(':', expand = True)[1]

df.MP = df.MP.replace('Did Not Play', np.nan).replace('Did Not Dress', np.nan).replace('Not With Team', np.nan).replace('Player Suspended', np.nan)

df['played'] = np.where(df['FG'].isna(), False, True)
df.MP = np.where(df['MP'] == 'NaT', 0, df['MP'])

In [34]:
df.to_csv('C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/merged_seasons_2014_2021.csv')

In [35]:
file_end = '_gamesbox_score.csv'
path = 'C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/'
df = box_score_table_reader(path, 2014, 2022, file_end)

In [36]:
df['OT_dummy'] = np.where(df.OT.isna(), False, True)
df['hour'] = df['hour'].str.replace('p', '')
df['date'] = pd.to_datetime(df.date, format = '%a, %b %d, %Y')
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year
df['day_name'] = df.date.dt.day_name()
df.att = pd.to_numeric(df.att.str.replace(',', ''))

In [37]:
df.dtypes

Unnamed: 0                 int64
date              datetime64[ns]
hour                      object
visitor                   object
visitor_id                object
visitor_score              int64
home                      object
home_id                   object
home_score                 int64
box_score_link            object
OT                        object
att                      float64
season                     int64
OT_dummy                    bool
day                        int64
month                      int64
year                       int64
day_name                  object
dtype: object

Let's convert the databases to the same level and calculate the following features:
- Record/Rank of the team
- Match-up results between the two teams if the teams have already played against one another this season
- Whether the team has played particularly well in the last ten games
- Team statistics such as assists per game, three points shots made per game, defensive blocks per game, et cetera
- Team record at home games, away games, and conference games

In [38]:
box_scores = pd.read_csv('C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/merged_seasons_2014_2021.csv')

In [57]:
df = df[(df.season == 2019) & (df.date < '2019-01-01')]

(547, 18)

In [65]:
df.box_score_link = df.box_score_link.str.rsplit('/').apply(lambda x: x[-1])

In [72]:
box_scores = pd.merge(box_scores, df.loc[:,['date', 'box_score_link']], left_on = 'game_id', right_on = 'box_score_link', how = 'left')

In [79]:
df['home_win'] = np.where(df.home_score > df.visitor_score, 1, 0)
df['visitor_win'] = np.where(df.home_score > df.visitor_score, 0, 1)

In [89]:
df.head()

,Unnamed: 0,date,hour,visitor,visitor_id,visitor_score,home,home_id,home_score,box_score_link,OT,att,season,OT_dummy,day,month,year,day_name,home_win
0,0,2013-10-29,8:00,Chicago Bulls,/teams/CHI/2014.html,95,Miami Heat,/teams/MIA/2014.html,107,201310290MIA.html,NaN,19964.0,2014,False,29,10,2013,Tuesday,1
1,1,2013-10-29,10:30,Los Angeles Clippers,/teams/LAC/2014.html,103,Los Angeles Lakers,/teams/LAL/2014.html,116,201310290LAL.html,NaN,18997.0,2014,False,29,10,2013,Tuesday,1
2,2,2013-10-30,7:00,Brooklyn Nets,/teams/BRK/2014.html,94,Cleveland Cavaliers,/teams/CLE/2014.html,98,201310300CLE.html,NaN,20562.0,2014,False,30,10,2013,Wednesday,1
3,3,2013-10-30,7:00,Boston Celtics,/teams/BOS/2014.html,87,Toronto Raptors,/teams/TOR/2014.html,93,201310300TOR.html,NaN,20155.0,2014,False,30,10,2013,Wednesday,1
4,4,2013-10-30,7:00,Miami Heat,/teams/MIA/2014.html,110,Philadelphia 76ers,/teams/PHI/2014.html,114,201310300PHI.html,NaN,19523.0,2014,False,30,10,2013,Wednesday,1


In [85]:
df[(df.visitor_id == '/teams/CHI/2014.html') | (df.home_id == '/teams/CHI/2014.html')].home_win.cumsum()

0        1
16       2
32       3
59       4
77       5
        ..
1235    47
1241    47
1249    47
1255    48
1262    48
Name: home_win, Length: 87, dtype: int32

In [ ]:
def record_counter(df, date, team_id_v, team_id_h):
    df = df[df.date < date]
    
    visitor_df = df[(df.visitor_id == team_id_v) | (df.home_id == team_id_v)]
    visitor_df['win_numb_v'] = visitor_df.home_win.cumsum()
    visitor_df['played_games_v'] = pd.Series([x for x in range(1, len(df) + 1)])
    visitor_df['win_streak'] = visitor_df..apply(lambda x:)
     
    home_df = df[(df.visitor_id == team_id_h) | (df.home_id == team_id_h)]
    home_df['win_numb_h'] = visitor_df.home_win.cumsum()
    home_df['played_games_h'] = pd.Series([x for x in range(1, len(df) + 1)])
    
    df['home_win_last_time'] =  np.where(df[((df.visitor_id == team_id_v) & (df.home_id == team_id_h)) | 
                                   ((df.visitor_id == team_id_h) & (df.home_id == team_id_v))])
    
    return home_df, visitor_df
    

In [88]:
pd.Series([x for x in range(1,len(df) + 1)])

0          1
1          2
2          3
3          4
4          5
        ... 
9649    9650
9650    9651
9651    9652
9652    9653
9653    9654
Length: 9654, dtype: int64

In [74]:
box_scores = box_scores[(df.season == 2019) & (df.date < '2019-01-01')]

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


ValueError: cannot reindex from a duplicate axis